# 사전 작업

## 모듈 로드

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
import gc

In [2]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold, StratifiedKFold

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
pd.set_option('display.max_columns', 600)

# 통합 데이터

In [5]:
def drop_features(data, keyword, debug=False):
    if debug:
        data = data[:1000]
        
    logits = []
    for col in data.columns:
        if keyword in col:
            logits.append(col)
        else:
            pass
    
    return logits

## 데이터 로드

In [6]:
path = './data/'

In [7]:
train = pd.read_csv(path + 'trainFE_0213.csv')
hist = pd.read_csv(path + 'histFE_0213.csv')
new = pd.read_csv(path + 'newFE_0213.csv')
test = pd.read_csv('./data_feature_engineering/train_test_v2.csv')
test = test[test.target.isnull()]

In [8]:
train = train.merge(hist, left_on='card_id', right_on='hist_card_id', how='left').merge(new, left_on='card_id', right_on='new_card_id', how='left')
train.drop(['hist_card_id', 'new_card_id'], axis = 1, inplace=True)

In [9]:
test = test.merge(hist, left_on='card_id', right_on='hist_card_id', how='left').merge(new, left_on='card_id', right_on='new_card_id', how='left')
test.drop(['hist_card_id', 'new_card_id'], axis = 1, inplace=True)

In [10]:
train.head(3)

,first_active_month,card_id,feature_1,feature_2,feature_3,target,first_active,first_active_year,first_active_quarter,first_active_weekofyear,first_active_dayofweek,first_active_elapsed_time_from_trade,first_active_elapsed_time_from_today,first_active_total_day,days_feature1_trade,days_feature2_trade,days_feature3_trade,days_feature1_trade_ratio,days_feature2_trade_ratio,days_feature3_trade_ratio,outliers,feature_1_outlier,feature_2_outlier,feature_3_outlier,first_active_year_outlier,first_active_month_outlier,first_active_total_day_outlier,feature_sum_outlier,feature_mean_outlier,feature_max_outlier,feature_min_outlier,feature_var_outlier,feature_sum,feature_mean,feature_max,feature_min,feature_var,hist_fromRefDate_sum,hist_fromRefDate_min,hist_fromRefDate_max,hist_fromRefDate_mean,hist_fromRefDate_median,hist_fromRefDate_var,hist_fromRefDate_quantileRange,hist_purchase_year_mode,hist_purchase_year_nunique,hist_purchase_year_mean,hist_purchase_year_max,hist_purchase_year_min,hist_purchase_month_mode,hist_purchase_month_nunique,hist_purchase_month_mean,hist_purchase_month_max,hist_purchase_month_min,hist_purchase_day_mode,hist_purchase_day_nunique,hist_purchase_day_mean,hist_purchase_day_var,hist_purchase_day_max,hist_purchase_day_min,hist_purchase_day_skew,hist_purchase_hour_mode,hist_purchase_hour_nunique,hist_purchase_hour_mean,hist_purchase_hour_var,hist_purchase_hour_max,hist_purchase_hour_min,hist_purchase_hour_skew,hist_purchase_date_max,hist_purchase_date_min,hist_month_lag_sum,hist_month_lag_mean,hist_month_lag_median,hist_month_lag_var,hist_month_lag_max,hist_month_lag_min,hist_month_lag_quantileRange,hist_month_lag_skew,hist_purchase_dayofweek_mode,hist_purchase_dayofweek_nunique,hist_purchase_dayofweek_mean,hist_purchase_dayofweek_max,hist_purchase_dayofweek_min,hist_purchase_weekofyear_mode,hist_purchase_weekofyear_nunique,hist_purchase_weekofyear_mean,hist_purchase_weekofyear_max,hist_purchase_weekofyear_min,hist_purchase_weekend_mode,hist_purchase_weekend_sum,hist_purchase_weekend_mean,hist_Christmas_Day_2017_mean,hist_Children_day_2017_mean,hist_Black_Friday_2017_mean,hist_Mothers_Day_2018_mean,hist_purchase_date_total_day_max,hist_purchase_date_total_day_min,hist_purchase_date_total_day_mean,hist_purchase_date_total_day_var,hist_purchase_date_total_day_skew,hist_month_diff_from_trade_max,hist_month_diff_from_trade_min,hist_month_diff_from_trade_mean,hist_month_diff_from_trade_var,hist_month_diff_from_trade_skew,hist_month_diff_from_today_max,hist_month_diff_from_today_min,hist_month_diff_from_today_mean,hist_month_diff_from_today_var,hist_month_diff_from_today_skew,hist_city_id_count,hist_city_id_mode,hist_city_id_nunique,hist_state_id_mode,hist_state_id_nunique,hist_subsector_id_mode,hist_subsector_id_nunique,hist_merchant_category_id_mode,hist_merchant_category_id_nunique,hist_merchant_id_mode,hist_merchant_id_nunique,hist_merchant_visit_sum,hist_merchant_visit_mean,hist_merchant_visit_min,hist_merchant_visit_max,hist_merchant_visit_nunique,hist_merchant_visit_size,hist_merchant_visit_mode,hist_merchant_try_mean,hist_merchant_try_std,hist_merchant_try_min,hist_merchant_try_max,hist_merchant_try_nunique,hist_merchant_try_size,hist_merchant_try_mode,hist_installments_sum,hist_installments_mean,hist_installments_median,hist_installments_var,hist_installments_max,hist_installments_min,hist_installments_quantileRange,hist_installments_null_cnt,hist_purchase_amount_sum,hist_purchase_amount_mean,hist_purchase_amount_median,hist_purchase_amount_var,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_quantileRange,hist_purchase_amount_skew,hist_purchase_amount_over_550,hist_purchase_amount_trim_sum,hist_purchase_amount_trim_mean,hist_purchase_amount_trim_var,hist_purchase_amount_trim_max,hist_purchase_amount_trim_min,hist_purchase_amount_trim_skew,hist_authorized_flag_mean,hist_authorized_flag_sum,hist_category_1_mode,hist_category_1_nunique,hist_category_1_mean,hist_category_1_var,hist_category_1_s

In [87]:
test.shape

(123623, 37)

In [88]:
train.shape

(201917, 282)

In [89]:
train.dtypes[train.dtypes == 'object']

card_id                   object
first_active              object
hist_purchase_date_max    object
hist_purchase_date_min    object
dtype: object

## 제거할 피처

In [90]:
FEATS_EXCLUDED = ['first_active', 'card_id', 'target', 'outliers',
                  'hist_purchase_date_max', 'hist_purchase_date_min', 
                  'new_purchase_date_max', 'new_purchase_date_min']

# FEATS_EXCLUDED += drop_features(train, 'hist_duration')
# FEATS_EXCLUDED += drop_features(train, 'hist_amount_month_ratio')
# FEATS_EXCLUDED += drop_features(train, 'new_duration')
# FEATS_EXCLUDED += drop_features(train, 'new_amount_month_ratio')
# FEATS_EXCLUDED += drop_features(train, 'hist_price')
# FEATS_EXCLUDED += drop_features(train, 'new_price')
# FEATS_EXCLUDED += drop_features(train, 'days_')
# FEATS_EXCLUDED += drop_features(train, 'feature_1_outlier')

In [91]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 30, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 4,
         "random_state": 4590}

In [92]:
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=15)

oof_lgb = np.zeros(len(train))
predictions_lgb = np.zeros(len(test))
feature_importance = pd.DataFrame()

train_columns = [f for f in train.columns if f not in FEATS_EXCLUDED]

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, train['outliers'].values)):    
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx][train_columns], label=train.iloc[trn_idx]['target'])
    val_data = lgb.Dataset(train.iloc[val_idx][train_columns], label=train.iloc[val_idx]['target'])

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 200)
    oof_lgb[val_idx] = clf.predict(train.iloc[val_idx][train_columns], num_iteration=clf.best_iteration)
    
    predictions_lgb += clf.predict(test[train_columns], num_iteration=clf.best_iteration) / folds.n_splits
    
    fold_importance = pd.DataFrame()
    fold_importance["Feature"] = train_columns
    fold_importance["importance"] = clf.feature_importance()
    fold_importance["fold"] = fold_ + 1
    feature_importance = pd.concat([feature_importance, fold_importance], axis=0)
    
print("CV score: {:<8.5f}".format(mean_squared_error(train.target.values, oof_lgb)**0.5))

fold n°0
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 3.68945	valid_1's rmse: 3.75943
[200]	training's rmse: 3.6156	valid_1's rmse: 3.74535
[300]	training's rmse: 3.56733	valid_1's rmse: 3.74161
[400]	training's rmse: 3.53406	valid_1's rmse: 3.74018
[500]	training's rmse: 3.50888	valid_1's rmse: 3.74003
[600]	training's rmse: 3.48589	valid_1's rmse: 3.73981
Early stopping, best iteration is:
[413]	training's rmse: 3.53055	valid_1's rmse: 3.73975


KeyError: "['hist_fromRefDate_sum' 'hist_fromRefDate_min' 'hist_fromRefDate_max'\n 'hist_fromRefDate_mean' 'hist_fromRefDate_median' 'hist_fromRefDate_var'\n 'hist_fromRefDate_quantileRange' 'hist_purchase_year_mode'\n 'hist_purchase_year_nunique' 'hist_purchase_year_mean'\n 'hist_purchase_year_max' 'hist_purchase_year_min'\n 'hist_purchase_month_mode' 'hist_purchase_month_nunique'\n 'hist_purchase_month_mean' 'hist_purchase_month_max'\n 'hist_purchase_month_min' 'hist_purchase_day_mode'\n 'hist_purchase_day_nunique' 'hist_purchase_day_mean'\n 'hist_purchase_day_var' 'hist_purchase_day_max' 'hist_purchase_day_min'\n 'hist_purchase_day_skew' 'hist_purchase_hour_mode'\n 'hist_purchase_hour_nunique' 'hist_purchase_hour_mean'\n 'hist_purchase_hour_var' 'hist_purchase_hour_max'\n 'hist_purchase_hour_min' 'hist_purchase_hour_skew' 'hist_month_lag_sum'\n 'hist_month_lag_mean' 'hist_month_lag_median' 'hist_month_lag_var'\n 'hist_month_lag_max' 'hist_month_lag_min' 'hist_month_lag_quantileRange

In [70]:
len(train.card_id)

201917

In [71]:
len(oof_lgb)

201917

In [66]:
print("CV score: {:<8.5f}".format(mean_squared_error(train.target.values, oof_lgb)**0.5))

CV score: 3.65209 


In [74]:
sub = pd.read_csv('./data/sample_submission.csv')

In [76]:
sub.target = predictions_lgb

In [78]:
sub.to_csv('./data/sub_new_fe.csv', index=False)

In [72]:
predictions_lgb

array([-2.50495246, -0.32532403, -0.81959737, ...,  0.8803522 ,
       -3.04385696, -0.03560506])